# 近似動的計画法

参考
* [RL Theory Lecture 8](https://rltheory.github.io/w2022-lecture-notes/planning-in-mdps/lec8/)

強化学習アルゴリズムは動的計画法にさまざまな近似（探索や関数近似など）を盛り込んだフレームワークと言えます。
よって、近似誤差が動的計画法に与える影響を解析することで、強化学習の近似誤差に対する脆弱性を見積もることができそうです。
このような解析手段は近似動的計画法として知られています。

## 近似方策反復法

**参考**
* [RL Theory Lecture 8](https://tadashik.github.io/rltheory-jp/lecture-notes/planning-in-mdps/lec8/)
* [Leverage the Average: an Analysis of KL Regularization in Reinforcement Learning](https://arxiv.org/abs/2003.14089)：表記を参考にしました

**表記**

* $q$についての貪欲方策：$\mathcal{G}(q) = \argmax_{\pi} \langle q, \pi \rangle$
* 方策による状態遷移：$P_\pi q = P \langle q, \pi \rangle$
* ベルマン期待作用素：$T_\pi q = r + \gamma P \langle q, \pi \rangle$
* ベルマン最適作用素：$T q = r + \gamma P \max_{\pi} \langle q, \pi \rangle$
* 報酬関数の範囲：$r \in [0, 1]^{S\times A}$

近似方策反復法は近似誤差が**方策反復法**の漸近的な挙動に与える影響を解析するためのフレームワークです。
近似方策反復法では次の更新を繰り返します。

$$
\begin{cases}
\pi_{k+1} \in \mathcal{G}(q_k) \\
q_{k+1} = (T_{\pi_{k+1}})^\infty q_k + \varepsilon_{k+1} = q_{\pi_{k+1}} + \varepsilon_{k+1}
\end{cases}
$$

ここで、$\varepsilon_k$はなんらかの近似誤差（例えばNNなどの近似誤差）を表しています。

### 誤差伝搬解析

近似方策反復法の$k$ステップ目の性能$\|q_* - q_{\pi_{k}}\|_\infty$は次のように誤差に影響されます。

$$\left\|q_*-q_{\pi_k}\right\|_{\infty} \leq \frac{\gamma^k}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq k-1}\left\|\varepsilon_s\right\|_{\infty}$$

これを示すために、次の**価値差分方程式**と**指数的改善定理**が必要になります。

---

**価値差分方程式**

$q_{\pi^{\prime}}=\left(I-\gamma P_{\pi^{\prime}}\right)^{-1} r$であるから、

$$\begin{aligned} q^{\pi^{\prime}}-q^\pi & =\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[r-\left(I-\gamma P_{\pi^{\prime}}\right) q^\pi\right] \\ & =\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[T_{\pi^{\prime}} q^\pi-q^\pi\right] .\end{aligned}$$

---

---

**指数的改善定理**

$\pi^\prime$を任意の方策として、$\varepsilon$を次で定義します。

$$T q_\pi=T_{\pi^{\prime}} q_\pi+\varepsilon$$

このとき、

$$
\left\|q_*-q_{\pi^{\prime}}\right\|_{\infty} \leq \gamma\left\|q_*-q_\pi\right\|_{\infty}+\frac{1}{1-\gamma}\|\varepsilon\|_{\infty}
$$

が成立します。証明してみましょう。　

まず、$T_{\pi_*} q_*=q_*$です。よって、

$$
\begin{aligned}
q_*-q_{\pi^{\prime}} & =T_{\pi_*} q_*-T_{\pi_*} q_\pi+\overbrace{T_{\pi_*} q_\pi}^{\leq T q_\pi}-T_{\pi^{\prime}} q_\pi+T_{\pi^{\prime}} q_\pi-T_{\pi^{\prime}} q_{\pi^{\prime}} \\
& \leq \gamma P_{\pi_*}\left(q_*-q_\pi\right)+\varepsilon+\gamma P_{\pi^{\prime}}\left(q_\pi-q_{\pi^{\prime}}\right)
\end{aligned}
$$

が成り立ちます。ここで、$T q_\pi=T_{\pi^{\prime}} q_\pi+\varepsilon$の定義を利用しています。

また、$q_\pi=T_\pi q^\pi \leq T q^\pi$と価値差分方程式を利用すると、

$$
q_\pi-q_{\pi^{\prime}}=\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[q_\pi-T_{\pi^{\prime}} q_\pi\right] \leq\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\left[T q_\pi-\left(T q_\pi-\varepsilon\right)\right]=\left(I-\gamma P_{\pi^{\prime}}\right)^{-1} \varepsilon
$$

が成立します。ここまでに求めた不等式二つを合体させて、

$$q_*-q_{\pi^{\prime}} \leq \gamma P_{\pi_*}\left(q_*-q_\pi\right)+\left(I-\gamma P_{\pi^{\prime}}\right)^{-1} \varepsilon$$

が得られます。あとは両辺で最大値ノルムをとり、三角不等式と$\left\|\left(I-\gamma P_{\pi^{\prime}}\right)^{-1}\right\|_{\infty} \leq 1 /(1-\gamma)$を使えば簡単です。

---

この指数的改善定理を使うと、次のように帰納法で誤差バウンドを示すことができます。

まず、$k=1$の時を考えましょう。指数的改善定理から、　

$$
\left\|q_*-q_{\pi_1}\right\| \leq \gamma\left\|q_*-q_{\pi_0}\right\|+\frac{1}{1-\gamma}\left\|\varepsilon_0\right\|_{\infty}
$$

が成り立ちます。報酬の最大値が１なので、$\|q_* - q_{\pi_0}\| \leq 1 / (1 - \gamma)$です。よって、

$$
\left\|q_*-q_{\pi_1}\right\| \leq \gamma\left\|q_*-q_{\pi_0}\right\|+\frac{1}{1-\gamma}\left\|\varepsilon_0\right\|_{\infty} \leq \frac{\gamma^1}{1-\gamma}+\frac{1}{(1-\gamma)^2}\left\|\varepsilon_0\right\|_{\infty}
$$

です。
つづいて、近似方策反復法の$k=i (i>1)$ステップ目の性能$\|q_* - q_{\pi_{k}}\|_\infty$が次のようにバウンドされていると仮定します。

$$\left\|q_*-q_{\pi_i}\right\|_{\infty} \leq \frac{\gamma^i}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}$$

このとき、

$$
\begin{aligned}
\left\|q_*-q_{i+1}\right\| & \leq \gamma\left\|q_*-q_{\pi_i}\right\|_{\infty}+\frac{1}{1-\gamma}\left\|\varepsilon_i\right\|_{\infty}\text { (指数的改善定理) } \\
& \leq \gamma\left(\frac{\gamma^i}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}\right)+\frac{1}{1-\gamma}\left\|\varepsilon_i\right\|_{\infty} \quad \text { (帰納法の仮定) } \\
& =\frac{\gamma^{i+1}}{1-\gamma}+\frac{\gamma}{(1-\gamma)^2} \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}+\frac{1}{1-\gamma}\left\|\varepsilon_i\right\|_{\infty} \\
& =\frac{\gamma^{i+1}}{1-\gamma}+\frac{1}{(1-\gamma)^2}\left(\gamma \max _{0 \leq s \leq i-1}\left\|\varepsilon_s\right\|_{\infty}+(1-\gamma)\left\|\varepsilon_i\right\|_{\infty}\right) \\
& \leq \frac{\gamma^{i+1}}{1-\gamma}+\frac{1}{(1-\gamma)^2}\left(\gamma \max _{0 \leq s \leq i}\left\|\varepsilon_s\right\|_{\infty}+(1-\gamma) \max _{0 \leq s \leq i}\left\|\varepsilon_s\right\|_{\infty}\right) \\
& =\frac{\gamma^{i+1}}{1-\gamma}+\frac{1}{(1-\gamma)^2} \max _{0 \leq s \leq i}\left\|\varepsilon_s\right\|_{\infty}
\end{aligned}
$$

以上で証明終わりです。

## $\lambda$-方策反復法

**参考**
* [Performance bound for Approximate Optimistic Policy Iteration](https://hal.inria.fr/inria-00480952/document)
* [Leverage the Average: an Analysis of KL Regularization in Reinforcement Learning](https://arxiv.org/abs/2003.14089)：表記を参考にしました

方策反復法では$(T_{\pi_{k+1}})^\infty$を使っていました。
一方で、Q学習や[マルチステップ方策評価](RL_multi_step.ipynb)のように、様々な形式の方策評価が存在します。
そこで、次のような**一般的な**近似方策反復法を考えます。

$$
\begin{cases}
\pi_{k+1} \in \mathcal{G}(q_k) \\
q_{k+1} = \sum_{n \geq 1} \lambda_n (T_{\pi_{k+1}})^n q_k + \epsilon_{k+1}
\end{cases}
$$

ここで、$\sum_{n>1} \lambda_n=1$はマルチステップ方策評価の重み係数です。詳しくは[マルチステップ方策評価](RL_multi_step.ipynb)を参考にしてください。
この誤差解析をしてみましょう。

### 誤差伝搬解析

---

**直感的な方針**

まず、$q_* - q_{\pi_k}=q_* - q_k + q_k - q_{\pi_k}$のように分解します。
そして「$q_k$, $q_{\pi_k}$, $q_*$はなんとなく近いだろう」という方針のもとで、$q_k - q_{\pi_k}$や$q_k - q_*$をバウンドしていきます。

---

まず、
$$
\begin{aligned}\left\|q_*-q_{\pi_k}\right\|_{\infty} & =\max \left(q_*-q_{\pi_k}\right) \\ & =\max \left(q_*-q_k+\epsilon_k+q_k-\epsilon_k-q_{\pi_k}\right) \\ & =\max \left(d_k+s_k\right) \\ & \leq \max d_k+\max s_k\end{aligned}
$$